# Watershed Delineation Methods

Run delineations for both the "series" and "parallel" approaches.

In [13]:
from pathlib import Path

from arcpy import FeatureSet
from arcpy.management import MakeFeatureLayer, SelectLayerByAttribute, GetCount

from arcpy.sa import Watershed
from arcpy.sa import FlowLength
from arcpy.sa import Raster

from arcpy.da import SearchCursor

from tqdm.notebook import tqdm

#import arcgis

In [2]:
wd = Path(r"D:\Dropbox (CivicMapper)\Projects\202004-02 Cornell Modeling\3 - Production\tool outputs")

inlets = wd / "c19 tests"  / "c19_snapped_points.shp"
dem = wd / "c19 tests" / "dem_clip_simple.tif"
fd = wd / "c19 tests" / "dem_clip_simple_flowdir.tif"

pour_point_field = "NAACC_ID"

In [3]:
fs = FeatureSet(str(inlets))

In [4]:
#w = Watershed(str(fd), str(inlets), pour_point_field)

In [6]:
out_layer = "inlets_layer"
fslyr = MakeFeatureLayer(
    in_features=fs, 
    out_layer=out_layer
)

In [17]:
int(GetCount(fs).getOutput(0))

'196'

In [30]:
sheds = []
with SearchCursor(fs, [pour_point_field]) as sc:
    for inlet in tqdm(sc, total=int(GetCount(fs).getOutput(0))):
        #out_layer = "inlet_{}".format(inlet[0])
        selected_inlet = SelectLayerByAttribute(
            in_layer_or_view=fslyr,
            selection_type='NEW_SELECTION',
            where_clause="{0}= {1}".format(pour_point_field, inlet[0])
        )
                
        one_shed = Watershed(
            in_flow_direction_raster=str(fd),
            in_pour_point_data=selected_inlet,
            pour_point_field=pour_point_field
        )
        sheds.append((inlet[0], one_shed))

  0%|          | 0/196 [00:00<?, ?it/s]

In [31]:
sheds[0]

(70978,
 D:\Dropbox (CivicMapper)\Projects\202004-02 Cornell Modeling\3 - Production\tool outputs\c19 tests\Watersh_dem_cli3.tif)